# Simple RNN Modles for Sequential MNIST with Tensorflow

Based on the work of [Aymeric Damien](https://github.com/aymericdamien/TensorFlow-Examples/) and [Sungjoon](https://github.com/sjchoi86/tensorflow-101/blob/master/notebooks/rnn_mnist_simple.ipynb)

## RNN Overview

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" alt="nn" style="width: 600px;"/>

References:
- [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf), Sepp Hochreiter & Jurgen Schmidhuber, Neural Computation 9(8): 1735-1780, 1997.

## MNIST Dataset Overview

This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

To classify images using a recurrent neural network, we consider every image row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then handle 28 sequences of 28 timesteps for every sample.

More info: http://yann.lecun.com/exdb/mnist/

## System Information

In [0]:
!pip install watermark

In [0]:
from pathlib import Path
import random 
from datetime import datetime

import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
%load_ext watermark
%watermark -v -m -p tensorflow,numpy -g

CPython 3.6.3
IPython 5.5.0

tensorflow 1.6.0
numpy 1.14.2

compiler   : GCC 7.2.0
system     : Linux
release    : 4.4.111+
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   :


## Row-by-row Sequential MNIST

How RNN model for row-by-row sequential MNIST works from [Sungjoon's notebook](https://github.com/sjchoi86/tensorflow-101/blob/master/notebooks/rnn_mnist_simple.ipynb):
![](https://raw.githubusercontent.com/sjchoi86/Tensorflow-101/582cc1d946f0ecbce078e493b8ccb1d7b40684df/notebooks/images/etc/rnn_mnist_look.jpg)

### Basic LSTM Model
Directly taken from [Aymeric Damien's notebook](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/recurrent_network.ipynb).

In [0]:
# Training Parameters
learning_rate = 0.001
training_steps = 5000
batch_size = 128
display_step = 200

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 64 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [0]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [0]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [0]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [0]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.8235, Training Accuracy= 0.102
Step 200, Minibatch Loss= 2.2561, Training Accuracy= 0.156
Step 400, Minibatch Loss= 2.1433, Training Accuracy= 0.219
Step 600, Minibatch Loss= 2.0626, Training Accuracy= 0.398
Step 800, Minibatch Loss= 2.0022, Training Accuracy= 0.344
Step 1000, Minibatch Loss= 1.9623, Training Accuracy= 0.359
Step 1200, Minibatch Loss= 1.9050, Training Accuracy= 0.398
Step 1400, Minibatch Loss= 1.8771, Training Accuracy= 0.344
Step 1600, Minibatch Loss= 1.8084, Training Accuracy= 0.375
Step 1800, Minibatch Loss= 1.6760, Training Accuracy= 0.477
Step 2000, Minibatch Loss= 1.7269, Training Accuracy= 0.453
Step 2200, Minibatch Loss= 1.6756, Training Accuracy= 0.516
Step 2400, Minibatch Loss= 1.6708, Training Accuracy= 0.398
Step 2600, Minibatch Loss= 1.6657, Training Accuracy= 0.469
Step 2800, Minibatch Loss= 1.4649, Training Accuracy= 0.523
Step 3000, Minibatch Loss= 1.4580, Training Accuracy= 0.617
Step 3200, Minibatch Loss= 1.4600, Training Acc

### Tidier Version of the LSTM Model

1. Use LSTMBlockCell, which should be faster than BasicLSTMCell
2. Replace manual weight definitions with tf.layers.Dense
3. Replace tf.nn.softmax_cross_entropy_with_logits with tf.nn.softmax_cross_entropy_with_logits_v2
4. Group graph definition together
5. Replace rnn.dynamic_rnn with rnn.static_rnn. (So no need to unstack the tensor.)
6. Add a batch_normalization layer between LSTM and Dense layers.
7. Add gradient clipping for RNN gradient
8. Add a checkpoint saver
9. Evaluate test accuracy every N steps (BAD PRACTICE: use a validation set instead)
10. Replace GradientDescentOptimizer with RMSPropOptimizer
11. Use tf.set_random_seed to control randomness

In [0]:
# Training Parameters
learning_rate = 0.02
training_steps = 5000
batch_size = 32
display_step = 250

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 28 # timesteps
num_hidden = 64 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

In [0]:
def RNN(x):
    # Define a lstm cell with tensorflow
    lstm_cell = rnn.LSTMBlockCell(
        num_hidden, forget_bias=1.0)

    # Get lstm cell output
    # outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    outputs, states = tf.nn.dynamic_rnn(
        cell=lstm_cell, inputs=x, time_major=False, dtype=tf.float32)
    
    output_layer = tf.layers.Dense(
        num_classes, activation=None, 
        kernel_initializer=tf.orthogonal_initializer()
    )
    return output_layer(tf.layers.batch_normalization(outputs[:, -1, :]))

In [0]:
# Need to clear the default graph before moving forward
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(1)
    # tf Graph input
    X = tf.placeholder("float", [None, timesteps, num_input])
    Y = tf.placeholder("float", [None, num_classes])
    logits = RNN(X)
    prediction = tf.nn.softmax(logits)

    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=Y))
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
    gvs = optimizer.compute_gradients(loss_op)
    capped_gvs = [
        (tf.clip_by_norm(grad, 2.), var) if not var.name.startswith("dense") else (grad, var)
        for grad, var in gvs]
    for _, var in gvs:
        if var.name.startswith("dense"):
            print(var.name)    
    train_op = optimizer.apply_gradients(capped_gvs)  

    # Evaluate model (with test logits, for dropout to be disabled)
    correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
    
    print("All parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.global_variables()]))
    print("Trainable parameters:", np.sum([np.product([xi.value for xi in x.get_shape()]) for x in tf.trainable_variables()]))    

Instructions for updating:
keep_dims is deprecated, use keepdims instead
dense/kernel:0
dense/bias:0
All parameters: 73886
Trainable parameters: 24586


In [0]:
# Start training
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
best_val_acc = 0.8
with tf.Session(graph=graph, config=config) as sess:
    # Run the initializer
    sess.run(init)
    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            # Calculate accuracy for 128 mnist test images
            test_len = 128
            test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
            test_label = mnist.test.labels[:test_len]
            val_acc = sess.run(accuracy, feed_dict={X: test_data, Y: test_label})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) + ", Test Accuracy= " + \
                  "{:.3f}".format(val_acc))
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                save_path = saver.save(sess, "/tmp/model.ckpt", global_step=step)
                print("Model saved in path: %s" % save_path)
    print("Optimization Finished!")

Step 1, Minibatch Loss= 2.2981, Training Accuracy= 0.156, Test Accuracy= 0.117
Step 250, Minibatch Loss= 0.3176, Training Accuracy= 0.906, Test Accuracy= 0.766
Step 500, Minibatch Loss= 0.3501, Training Accuracy= 0.844, Test Accuracy= 0.883
Model saved in path: /tmp/model.ckpt-500
Step 750, Minibatch Loss= 0.0230, Training Accuracy= 1.000, Test Accuracy= 0.945
Model saved in path: /tmp/model.ckpt-750
Step 1000, Minibatch Loss= 0.0212, Training Accuracy= 1.000, Test Accuracy= 0.945
Step 1250, Minibatch Loss= 0.1140, Training Accuracy= 0.969, Test Accuracy= 0.906
Step 1500, Minibatch Loss= 0.0110, Training Accuracy= 1.000, Test Accuracy= 0.945
Step 1750, Minibatch Loss= 0.0060, Training Accuracy= 1.000, Test Accuracy= 0.961
Model saved in path: /tmp/model.ckpt-1750
Step 2000, Minibatch Loss= 0.0418, Training Accuracy= 1.000, Test Accuracy= 0.922
Step 2250, Minibatch Loss= 0.0168, Training Accuracy= 1.000, Test Accuracy= 0.969
Model saved in path: /tmp/model.ckpt-2250
Step 2500, Minibatch

## Pixel-by-pixel Sequential MNIST

View every example as a 784 x 1 sequence.

### CudnnGRU Model

Cudnn's implementation of GRU is much faster, but does not support variational dropout. It also does not support CPU mode.

Also some new additions:

1. Use tf.summary to save logs for Tensorboard
2. Use tf.variable_scope to group variables and operations
3. Use AdamOptimizer instead of RMSPropOptimizer (latter has some problem with CudnnGRU)

In [0]:
# Training Parameters
learning_rate = 0.002
training_steps = 5000
batch_size = 32
display_step = 250
total_batch = int(mnist.train.num_examples / batch_size)
print("Total number of batches:", total_batch)

# Network Parameters
num_input = 1 # MNIST data input (img shape: 28*28)
timesteps = 28 * 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

Total number of batches: 1718


In [0]:
def RNN(x):    
    gru = tf.contrib.cudnn_rnn.CudnnGRU(
        1, num_hidden,
        kernel_initializer=tf.orthogonal_initializer())
    outputs, _ = gru(tf.transpose(x, (1, 0, 2)))    
    output_layer = tf.layers.Dense(
        num_classes, activation=None, 
        kernel_initializer=tf.orthogonal_initializer(),
        trainable=True
    )
    # Linear activation, using rnn inner loop last output
    return output_layer(tf.layers.batch_normalization(outputs[-1, :, :]))

In [0]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(10)
    # tf Graph input
    X = tf.placeholder("float", [None, timesteps, num_input])
    Y = tf.placeholder("float", [None, num_classes])
    # Define weights
    logits = RNN(X)
    prediction = tf.nn.softmax(logits)

    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=Y))
    
    with tf.variable_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.1)
        gvs = optimizer.compute_gradients(loss_op)
        capped_gvs = [
            (tf.clip_by_norm(grad, 1.), var) if not var.name.startswith("dense") else (grad, var)
            for grad, var in gvs]
        for _, var in gvs:
            if var.name.startswith("dense"):
                print(var.name)
        train_op = optimizer.apply_gradients(capped_gvs)  
    
    with tf.variable_scope('summarize_gradients'):
        for grad, var in gvs:
            norm = tf.norm(tf.clip_by_norm(grad, 10.), ord=2)
            tf.summary.histogram(var.name.replace(":", "_") + '/gradient_l2', 
                                 tf.where(tf.is_nan(norm), tf.zeros_like(norm), norm))
        for grad, var in capped_gvs:
            norm = tf.norm(grad, ord=2)
            tf.summary.histogram(var.name.replace(":", "_") + '/gradient_clipped_l2', 
                                 tf.where(tf.is_nan(norm), tf.zeros_like(norm), norm))

    merged_summary_op = tf.summary.merge_all()
    
    with tf.variable_scope('evaluate'):
      # Evaluate model (with test logits, for dropout to be disabled)
      correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
      accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

dense/kernel:0
dense/bias:0


In [0]:
# Start training
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
best_val_acc = 0.8
log_dir = "logs/cudnngru/%s" % datetime.now().strftime("%Y%m%d_%H%M")
Path(log_dir).mkdir(exist_ok=True, parents=True)
tb_writer = tf.summary.FileWriter(log_dir, graph)
with tf.Session(graph=graph, config=config) as sess:
    # Run the initializer
    sess.run(init)
    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc, summary = sess.run(
                [loss_op, accuracy, merged_summary_op], 
                feed_dict={X: batch_x, Y: batch_y})
            tb_writer.add_summary(summary, global_step=step)
            tb_writer.flush()
            # Calculate accuracy for 128 mnist test images
            test_len = 128
            test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
            test_label = mnist.test.labels[:test_len]
            val_acc = sess.run(accuracy, feed_dict={X: test_data, Y: test_label})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) + ", Test Accuracy= " + \
                  "{:.3f}".format(val_acc))
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                save_path = saver.save(sess, "/tmp/model.ckpt", global_step=step)
                print("Model saved in path: %s" % save_path)
    print("Optimization Finished!")

Step 1, Minibatch Loss= 2.2955, Training Accuracy= 0.156, Test Accuracy= 0.117
Step 250, Minibatch Loss= 2.2108, Training Accuracy= 0.125, Test Accuracy= 0.117
Step 500, Minibatch Loss= 0.8783, Training Accuracy= 0.781, Test Accuracy= 0.641
Step 750, Minibatch Loss= 0.4712, Training Accuracy= 0.906, Test Accuracy= 0.734
Step 1000, Minibatch Loss= 0.2107, Training Accuracy= 0.969, Test Accuracy= 0.812
Model saved in path: /tmp/model.ckpt-1000
Step 1250, Minibatch Loss= 0.4126, Training Accuracy= 0.875, Test Accuracy= 0.883
Model saved in path: /tmp/model.ckpt-1250
Step 1500, Minibatch Loss= 0.1344, Training Accuracy= 0.969, Test Accuracy= 0.859
Step 1750, Minibatch Loss= 0.2449, Training Accuracy= 0.875, Test Accuracy= 0.898
Model saved in path: /tmp/model.ckpt-1750
Step 2000, Minibatch Loss= 0.1204, Training Accuracy= 0.969, Test Accuracy= 0.938
Model saved in path: /tmp/model.ckpt-2000
Step 2250, Minibatch Loss= 0.1673, Training Accuracy= 0.938, Test Accuracy= 0.930
Step 2500, Minibat

## Permuted Pixel-by-pixel Sequential MNIST

Increase the difficulty by shuffling the order of the sequence (by applying the same reindexing operation for all sequences).

### CudnnGRU Model
Basically the same. Just added a permutation operation in the graph definition.

In [0]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(10)
    # tf Graph input
    X_ = tf.placeholder("float", [None, timesteps, num_input])
    Y = tf.placeholder("float", [None, num_classes])
    
    # Permute the time steps
    np.random.seed(100)
    permute = np.random.permutation(784)
    X = tf.gather(X_, permute, axis=1)   
    
    # Define weights
    logits = RNN(X)
    prediction = tf.nn.softmax(logits)

    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=Y))
    
    with tf.variable_scope('optimizer'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.1)
        gvs = optimizer.compute_gradients(loss_op)
        capped_gvs = [
            (tf.clip_by_norm(grad, 1.), var) if not var.name.startswith("dense") else (grad, var)
            for grad, var in gvs]
        for _, var in gvs:
            if var.name.startswith("dense"):
                print(var.name)
        train_op = optimizer.apply_gradients(capped_gvs)  
    
    with tf.variable_scope('summarize_gradients'):
        for grad, var in gvs:
            norm = tf.norm(tf.clip_by_norm(grad, 10.), ord=2)
            tf.summary.histogram(var.name.replace(":", "_") + '/gradient_l2', 
                                 tf.where(tf.is_nan(norm), tf.zeros_like(norm), norm))
        for grad, var in capped_gvs:
            norm = tf.norm(grad, ord=2)
            tf.summary.histogram(var.name.replace(":", "_") + '/gradient_clipped_l2', 
                                 tf.where(tf.is_nan(norm), tf.zeros_like(norm), norm))

    merged_summary_op = tf.summary.merge_all()
    
    with tf.variable_scope('evaluate'):
      # Evaluate model (with test logits, for dropout to be disabled)
      correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
      accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

dense/kernel:0
dense/bias:0


In [0]:
#@title Default title text
# Start training
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
best_val_acc = 0.8
log_dir = "logs/cudnngru/%s" % datetime.now().strftime("%Y%m%d_%H%M")
Path(log_dir).mkdir(exist_ok=True, parents=True)
tb_writer = tf.summary.FileWriter(log_dir, graph)
with tf.Session(graph=graph, config=config) as sess:
    # Run the initializer
    sess.run(init)
    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X_: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc, summary = sess.run(
                [loss_op, accuracy, merged_summary_op], 
                feed_dict={X_: batch_x, Y: batch_y})
            tb_writer.add_summary(summary, global_step=step)
            tb_writer.flush()
            # Calculate accuracy for 128 mnist test images
            test_len = 128
            test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
            test_label = mnist.test.labels[:test_len]
            val_acc = sess.run(accuracy, feed_dict={X_: test_data, Y: test_label})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc) + ", Test Accuracy= " + \
                  "{:.3f}".format(val_acc))
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                save_path = saver.save(sess, "/tmp/model.ckpt", global_step=step)
                print("Model saved in path: %s" % save_path)
    print("Optimization Finished!")

Step 1, Minibatch Loss= 2.2866, Training Accuracy= 0.188, Test Accuracy= 0.078
Step 250, Minibatch Loss= 1.5686, Training Accuracy= 0.500, Test Accuracy= 0.508
Step 500, Minibatch Loss= 0.8842, Training Accuracy= 0.781, Test Accuracy= 0.555
Step 750, Minibatch Loss= 0.9858, Training Accuracy= 0.719, Test Accuracy= 0.648
Step 1000, Minibatch Loss= 0.9501, Training Accuracy= 0.750, Test Accuracy= 0.648
Step 1250, Minibatch Loss= 0.7780, Training Accuracy= 0.688, Test Accuracy= 0.688
Step 1500, Minibatch Loss= 1.0259, Training Accuracy= 0.688, Test Accuracy= 0.680
Step 1750, Minibatch Loss= 0.5693, Training Accuracy= 0.906, Test Accuracy= 0.781
Step 2000, Minibatch Loss= 0.6409, Training Accuracy= 0.844, Test Accuracy= 0.805
Model saved in path: /tmp/model.ckpt-2000
Step 2250, Minibatch Loss= 0.6085, Training Accuracy= 0.812, Test Accuracy= 0.719
Step 2500, Minibatch Loss= 0.5675, Training Accuracy= 0.812, Test Accuracy= 0.750
Step 2750, Minibatch Loss= 0.4034, Training Accuracy= 0.938, T